# Hyperparameter Tuning

These are the hyperparameters we would be tuning for getting the best results from the model:

1) Learning Rate \\
2) Batch Size' \\
3) L1 lambda \\
4) optimizer \\
6) Weight Decay \\
7) Epochs

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
import os
from PIL import Image
from tqdm import tqdm
from torchvision.utils import save_image
# local module imports
import config
from utils import save_checkpoint, load_checkpoint, save_some_examples
from UvU_Net_Generator import OuterUNet as generator
from Pix_Discriminator import Discriminator

torch.backends.cudnn.benchmark = True

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


torch.Size([1, 3, 512, 512])


In [2]:
def train_fn(
    disc, gen, loader, opt_disc, opt_gen, l1_loss, bce, g_scaler, d_scaler,
):
    loop = tqdm(loader, leave=True)

    for idx, (x, y) in enumerate(loop):
        x = x.to(config.DEVICE)
        y = y.to(config.DEVICE)

        # Train Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake = disc(x, y_fake.detach())
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2

        disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1_loss(y_fake, y) * config.L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        if idx % 10 == 0:
            loop.set_postfix(
                D_real=torch.sigmoid(D_real).mean().item(),
                D_fake=torch.sigmoid(D_fake).mean().item(),
            )

In [3]:
!unzip -q Sample_dataset.zip

In [4]:
class PairedImageDataset(Dataset):
    def __init__(self, input_dir, target_dir, transform=None):
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))
        self.transform = transform

        assert len(self.input_images) == len(self.target_images), "Mismatch between input and target images!"

    def __len__(self):
        return len(self.input_images)

    def __getitem__(self, idx):
        input_image_path = os.path.join(self.input_dir, self.input_images[idx])
        target_image_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_image_path).convert("RGB")
        target_image = Image.open(target_image_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        return input_image, target_image


input_dir = '/content/Sample_dataset/sobel_images'
target_dir = '/content/Sample_dataset/input_images'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

dataset = PairedImageDataset(input_dir=input_dir, target_dir=target_dir, transform=transform)

# Split dataset into training and validation (80-20 split)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader for training and validation
batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Example to iterate over the data loader
for batch_idx, (input_images, target_images) in enumerate(train_loader):
    print(f"Batch {batch_idx + 1}")
    print(f"Input batch size: {input_images.shape}")
    print(f"Target batch size: {target_images.shape}")


Batch 1
Input batch size: torch.Size([16, 3, 256, 256])
Target batch size: torch.Size([16, 3, 256, 256])
Batch 2
Input batch size: torch.Size([4, 3, 256, 256])
Target batch size: torch.Size([4, 3, 256, 256])


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
save_dir = '/content/drive/MyDrive/model_checkpoints/'  # Change this to your desired directory
os.makedirs(save_dir, exist_ok=True)
directory = '/content/evaluation/'
if not os.path.exists(directory):
    os.makedirs(directory)  # Create the directory if it doesn't exist

def main():
    disc = Discriminator(in_channels=3).to(config.DEVICE)
    gen = generator(in_channels=3, features=64).to(config.DEVICE)
    opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
    BCE = nn.BCEWithLogitsLoss()
    L1_LOSS = nn.L1Loss()

    if config.LOAD_MODEL:
        load_checkpoint(
            config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE,
        )
        load_checkpoint(
            config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE,
        )

    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()

    for epoch in range(config.NUM_EPOCHS):
        train_fn(
            disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler,
        )

        # Save models every 50 epochs to Google Drive
        if epoch % 50 == 49:
            print(f"Saving model at epoch {epoch}")
            torch.save({
                'epoch': epoch,
                'generator_state_dict': gen.state_dict(),
                'optimizer_gen_state_dict': opt_gen.state_dict(),
            }, f"{save_dir}generator_epoch_{epoch}.pth")

            torch.save({
                'epoch': epoch,
                'discriminator_state_dict': disc.state_dict(),
                'optimizer_disc_state_dict': opt_disc.state_dict(),
            }, f"{save_dir}discriminator_epoch_{epoch}.pth")

            save_some_examples(gen, val_loader, epoch, folder="evaluation")


if __name__ == "__main__":
    main()


<ipython-input-6-c9066b09f306>:23: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
<ipython-input-6-c9066b09f306>:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-2-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-2-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  6.16it/s, D_fake=0.154, D_real=0.784]


Saving model at epoch 49


100%|██████████| 2/2 [00:00<00:00,  6.02it/s, D_fake=0.0694, D_real=0.947]


Saving model at epoch 99


100%|██████████| 2/2 [00:00<00:00,  6.08it/s, D_fake=0.053, D_real=0.964]


...

Saving model at epoch 399


100%|██████████| 2/2 [00:00<00:00,  5.92it/s, D_fake=0.00478, D_real=0.997]


Saving model at epoch 449


100%|██████████| 2/2 [00:00<00:00,  5.51it/s, D_fake=0.00557, D_real=0.998]


Saving model at epoch 499


In [7]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00


In [8]:
import numpy as np
import random
import itertools
import torch

def calculate_validation_loss(disc, gen, val_loader, L1_LOSS, BCE, device):
    gen.eval()
    total_val_loss = 0
    total_batches = len(val_loader)

    with torch.no_grad():
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)

            y_fake = gen(x)

            D_fake = disc(x, y_fake)
            G_fake_loss = BCE(D_fake, torch.ones_like(D_fake))

            L1_loss = L1_LOSS(y_fake, y) * config.L1_LAMBDA

            val_loss = G_fake_loss + L1_loss

            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / total_batches
    return avg_val_loss

def train_and_validate(disc, gen, train_loader, val_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler, num_epochs):
    for epoch in range(num_epochs):
        gen.train()
        disc.train()
        train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)

    val_loss = calculate_validation_loss(disc, gen, val_loader, L1_LOSS, BCE, config.DEVICE)

    return val_loss

def randomized_search(param_dist, n_trials=10):
    best_params = None
    best_val_loss = float('inf')

    for trial in range(n_trials):
        params = {key: random.choice(value) for key, value in param_dist.items()}
        print(f"Trial {trial + 1} with parameters: {params}")

        learning_rate = params['learning_rate']
        batch_size = params['batch_size']
        L1_LAMBDA = params['L1_LAMBDA']
        LAMBDA_GP = params['LAMBDA_GP']
        optimizer_type = params['optimizer']
        betas = params['betas']
        weight_decay = params['weight_decay']
        num_epochs = params['num_epochs']

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        disc = Discriminator(in_channels=3).to(config.DEVICE)
        gen = generator(in_channels=3, features=64).to(config.DEVICE)

        if optimizer_type == 'Adam':
            opt_disc = optim.Adam(disc.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
            opt_gen = optim.Adam(gen.parameters(), lr=learning_rate, betas=betas, weight_decay=weight_decay)
        elif optimizer_type == 'RMSprop':
            opt_disc = optim.RMSprop(disc.parameters(), lr=learning_rate, weight_decay=weight_decay)
            opt_gen = optim.RMSprop(gen.parameters(), lr=learning_rate, weight_decay=weight_decay)

        BCE = nn.BCEWithLogitsLoss()
        L1_LOSS = nn.L1Loss()
        g_scaler = torch.cuda.amp.GradScaler()
        d_scaler = torch.cuda.amp.GradScaler()

        val_loss = train_and_validate(disc, gen, train_loader, val_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler, num_epochs)

        print(f"Validation Loss for Trial {trial + 1}: {val_loss}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_params = params

    return best_params, best_val_loss

if __name__ == "__main__":
    param_dist = {
        'learning_rate': [1e-5, 1e-4, 2e-4, 1e-3],
        'batch_size': [8, 16, 32],
        'L1_LAMBDA': [50, 100, 150],
        'LAMBDA_GP': [5, 10, 15],
        'optimizer': ['Adam', 'RMSprop'],
        'betas': [(0.5, 0.999), (0.9, 0.999)],
        'weight_decay': [0, 1e-5, 1e-4],
        'num_epochs': [300, 500]
    }

    best_params, best_val_loss = randomized_search(param_dist, n_trials=20)
    print(f"Best parameters: {best_params}, Best validation loss: {best_val_loss}")


Trial 1 with parameters: {'learning_rate': 0.0002, 'batch_size': 16, 'L1_LAMBDA': 150, 'LAMBDA_GP': 10, 'optimizer': 'Adam', 'betas': (0.9, 0.999), 'weight_decay': 0, 'num_epochs': 300}


<ipython-input-8-2c51e3123ebd>:85: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  g_scaler = torch.cuda.amp.GradScaler()
<ipython-input-8-2c51e3123ebd>:86: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  d_scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/2 [00:00<?, ?it/s]<ipython-input-2-6490c8d8cca4>:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
<ipython-input-2-6490c8d8cca4>:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 2/2 [00:00<00:00,  5.69it/s, D_fake=0.0388, D_real=0.931]


Validation Loss for Trial 1: 39.95629119873047
Trial 2 with parameters: {'learning_rate': 0.0002, 'batch_size': 16, 'L1_LAMBDA': 50, 'LAMBDA_GP': 10, 'optimizer': 'RMSprop', 'betas': (0.5, 0.999), 'weight_decay': 0, 'num_epochs': 300}


100%|██████████| 2/2 [00:00<00:00,  5.82it/s, D_fake=0.0219, D_real=0.973]


Validation Loss for Trial 2: 33.384403228759766
Trial 3 with parameters: {'learning_rate': 0.001, 'batch_size': 16, 'L1_LAMBDA': 50, 'LAMBDA_GP': 10, 'optimizer': 'RMSprop', 'betas': (0.9, 0.999), 'weight_decay': 0.0001, 'num_epochs': 300}


...

Validation Loss for Trial 18: 38.9878044128418
Trial 19 with parameters: {'learning_rate': 0.0002, 'batch_size': 16, 'L1_LAMBDA': 100, 'LAMBDA_GP': 10, 'optimizer': 'Adam', 'betas': (0.9, 0.999), 'weight_decay': 0.0001, 'num_epochs': 300}


100%|██████████| 2/2 [00:00<00:00,  5.49it/s, D_fake=0.00221, D_real=0.998]


Validation Loss for Trial 19: 34.73692321777344
Trial 20 with parameters: {'learning_rate': 1e-05, 'batch_size': 16, 'L1_LAMBDA': 50, 'LAMBDA_GP': 10, 'optimizer': 'Adam', 'betas': (0.5, 0.999), 'weight_decay': 0, 'num_epochs': 500}


100%|██████████| 2/2 [00:00<00:00,  5.32it/s, D_fake=0.0646, D_real=0.933]

Validation Loss for Trial 20: 29.038705825805664
Best parameters: {'learning_rate': 0.001, 'batch_size': 16, 'L1_LAMBDA': 150, 'LAMBDA_GP': 10, 'optimizer': 'Adam', 'betas': (0.5, 0.999), 'weight_decay': 0, 'num_epochs': 500}, Best validation loss: 26.840585708618164


Best parameters:

 {'learning_rate': 0.001, \\
  'batch_size': 16, \\
  'L1_LAMBDA': 150, \\
  'LAMBDA_GP': 10, \\
  'optimizer': 'Adam', \\
  'betas': (0.5, 0.999), \\
  'weight_decay': 0, \\
  'num_epochs': 500}

Best validation loss: 26.840585708618164
